<a href="https://colab.research.google.com/github/Arunsh-Data/ArunshAI.github.io/blob/main/Project_Sales_Dashboard_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [49]:
import pandas as pd


In [50]:
url = '/content/sample_data/Sales Dashboard Dataset_01.xlsx'
df = pd.read_excel(url)

print(df.head())

   Order ID Customer Last Name Customer First Name  Customer ID  \
0      3001             Hughes               Lewis         1234   
1      3002              Gomez                 Ali         1235   
2      3003               Beck                Mila         1236   
3      3004             Juarez             Ryleigh         1237   
4      3005            Preston            Leonidas         1238   

            Service Category Order Date  Quantity Ordered    Order Status  \
0             Interview Prep 2024-05-06                 3       Delivered   
1  Resume + LinkedIn Profile 2024-07-18                 3     In Progress   
2           LinkedIn profile 2024-07-14                 1       Delivered   
3           LinkedIn profile 2024-06-26                 2       Delivered   
4        Networking Strategy 2024-11-28                 1  Service Booked   

  State  Margins  Service Provider Expense  Sale Price   Total Profit (GMROI)  \
0    WA     0.10                     56.40          5

In [51]:
xls = pd.ExcelFile(url)
xls.sheet_names

['Sales', 'Customers', 'Grad_Careers_Services']

In [52]:
# Select the Sales sheet
sales_df = xls.parse('Sales')

# Perform EDA on the sales sheet
print("\n--- EDA for Sales Sheet ---")
print("\nShape:")
print(sales_df.shape)
print("\nInfo:")
print(sales_df.info())
print("\nDescribe:")
print(sales_df.describe())
print("\nMissing values:")
print(sales_df.isnull().sum())

# Select the Customer sheet
customer_df = xls.parse('Customers')

# Perform EDA on the customer sheet
print("\n--- EDA for Customers Sheet ---")
print("\nShape:")
print(customer_df.shape)
print("\nInfo:")
print(customer_df.info())
print("\nDescribe:")
print(customer_df.describe())
print("\nMissing values:")
print(customer_df.isnull().sum())


--- EDA for Sales Sheet ---

Shape:
(1000, 14)

Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 14 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   Order ID                  1000 non-null   int64         
 1   Customer Last Name        1000 non-null   object        
 2   Customer First Name       1000 non-null   object        
 3   Customer ID               1000 non-null   int64         
 4   Service Category          1000 non-null   object        
 5   Order Date                1000 non-null   datetime64[ns]
 6   Quantity Ordered          1000 non-null   int64         
 7   Order Status              1000 non-null   object        
 8   State                     1000 non-null   object        
 9   Margins                   1000 non-null   float64       
 10  Service Provider Expense  1000 non-null   float64       
 11  Sale Price                10

In [53]:
# Merge first and last name in both dataframes
sales_df['Full Name'] = sales_df['Customer First Name'] + ' ' + sales_df['Customer Last Name']
customer_df['Full Name'] = customer_df['Customer First Name'] + ' ' + customer_df['Customer Last Name']

# Drop the individual name columns
sales_df = sales_df.drop(['Customer First Name', 'Customer Last Name'], axis=1)
customer_df = customer_df.drop(['Customer First Name', 'Customer Last Name'], axis=1)

print("\nSales Sheet after merging names and dropping columns:")
print(sales_df.head())

print("\nCustomers Sheet after merging names and dropping columns:")
print(customer_df.head())


Sales Sheet after merging names and dropping columns:
   Order ID  Customer ID           Service Category Order Date  \
0      3001         1234             Interview Prep 2024-05-06   
1      3002         1235  Resume + LinkedIn Profile 2024-07-18   
2      3003         1236           LinkedIn profile 2024-07-14   
3      3004         1237           LinkedIn profile 2024-06-26   
4      3005         1238        Networking Strategy 2024-11-28   

   Quantity Ordered    Order Status State  Margins  Service Provider Expense  \
0                 3       Delivered    WA     0.10                     56.40   
1                 3     In Progress   ACT     0.30                    233.10   
2                 1       Delivered   NSW     0.25                    457.25   
3                 2       Delivered   VIC     0.25                     56.75   
4                 1  Service Booked   QLD     0.20                    271.00   

   Sale Price   Total Profit (GMROI) Sales Agent         Full Name 

In [54]:
# Check for duplicate rows in the Sales sheet
print("\nDuplicate rows in Sales sheet:")
print(sales_df.duplicated().sum())

# Check for duplicate rows in the Customers sheet
print("\nDuplicate rows in Customers sheet:")
print(customer_df.duplicated().sum())


Duplicate rows in Sales sheet:
0

Duplicate rows in Customers sheet:
0


In [55]:

# Identify common columns
common_columns = list(set(sales_df.columns) & set(customer_df.columns))
print(f"\nCommon columns: {common_columns}")

# Check if values in common columns are similar (optional, can be done with more specific checks)
# For simplicity here, we just check if there are any common columns to merge on.
if not common_columns:
    print("No common columns found for merging.")
else:
    # Check for potential overlapping or duplicates in common columns across dataframes
    # This is a simplified check; more robust checks depend on the specific data and columns
    for col in common_columns:
        overlap = pd.merge(sales_df[[col]], customer_df[[col]], on=col, how='inner')
        if not overlap.empty:
            print(f"\nOverlap detected in column '{col}'. Investigate data for potential issues.")

    # Merge the dataframes on all common columns
    # Since we are merging on all common columns, we don't need to explicitly handle 'x' and 'y' suffixes
    # as long as the values align.
    merged_df = pd.merge(sales_df, customer_df, on=common_columns, how='outer', suffixes=('', '_customer')) # Using outer join to keep all data

    # If you want to remove the automatically added suffixes for customer columns that were not common
    # but had the same name as a sales column, you can rename them.
    # Identify columns that got the '_customer' suffix
    customer_suffixed_cols = [col for col in merged_df.columns if col.endswith('_customer')]

    # You might want to inspect these columns to decide how to handle them (e.g., keep, drop, combine)
    print(f"\nColumns with '_customer' suffix after merging: {customer_suffixed_cols}")

    # Example: Removing the suffix if you are confident these columns can be treated identically
    # Note: This is a potentially lossy operation if the columns represent different data despite the name similarity
    # Consider a more sophisticated approach if necessary.
    # merged_df.columns = merged_df.columns.str.replace('_customer$', '', regex=True) # Uncomment to remove suffix

    print("\n--- Merged DataFrame Head ---")
    print(merged_df.head())
    print("\n--- Merged DataFrame Info ---")
    print(merged_df.info())


Common columns: ['Service Category', 'State', 'Order Status', 'Order Date', 'Sale Price ', 'Quantity Ordered', 'Customer ID', 'Full Name']

Overlap detected in column 'Service Category'. Investigate data for potential issues.

Overlap detected in column 'State'. Investigate data for potential issues.

Overlap detected in column 'Order Status'. Investigate data for potential issues.

Overlap detected in column 'Order Date'. Investigate data for potential issues.

Overlap detected in column 'Sale Price '. Investigate data for potential issues.

Overlap detected in column 'Quantity Ordered'. Investigate data for potential issues.

Overlap detected in column 'Customer ID'. Investigate data for potential issues.

Overlap detected in column 'Full Name'. Investigate data for potential issues.

Columns with '_customer' suffix after merging: []

--- Merged DataFrame Head ---
   Order ID  Customer ID Service Category Order Date  Quantity Ordered  \
0       NaN         1400   Interview Prep 2024

In [56]:
# Identify date columns - Assuming columns with 'Date' in their name are date columns
date_columns = [col for col in merged_df.columns if 'Date' in col]

# Iterate through identified date columns and format them
for col in date_columns:
    # Ensure the column is in datetime format first (important if not already)
    merged_df[col] = pd.to_datetime(merged_df[col], errors='coerce')
    # Format the date column to 'mm-year'
    merged_df[col] = merged_df[col].dt.strftime('%m-%Y')

# Display the head of the DataFrame with formatted date columns
print("\n--- Merged DataFrame with formatted date columns ---")
print(merged_df.head())

# Display info to confirm the data type (will likely be 'object' after formatting)
print("\nInfo after formatting date columns:")
print(merged_df.info())


--- Merged DataFrame with formatted date columns ---
   Order ID  Customer ID Service Category Order Date  Quantity Ordered  \
0       NaN         1400   Interview Prep    01-2024                 2   
1    3167.0         1400   Interview Prep    01-2024                 2   
2    3024.0         1257   Interview Prep    01-2024                 2   
3       NaN         1257   Interview Prep    01-2024                 3   
4       NaN         2092   Interview Prep    01-2024                 2   

  Order Status State  Margins  Service Provider Expense  Sale Price   \
0    Cancelled   ACT      NaN                       NaN          500   
1    Cancelled   ACT      0.1                     175.1         1751   
2    Cancelled   ACT      0.1                      66.6          666   
3    Cancelled   ACT      NaN                       NaN          750   
4    Delivered   ACT      NaN                       NaN          500   

   Total Profit (GMROI) Sales Agent         Full Name               

# Calculate Key Performance Indicators (KPIs)

In [57]:
# Calculate Total Revenue
total_revenue = merged_df['Sale Price '].sum()
print(f"Total Revenue: ${total_revenue:,.2f}")

# Calculate Total Profit
total_profit = merged_df['Total Profit (GMROI)'].sum()
print(f"Total Profit: ${total_profit:,.2f}")

# Calculate Number of Orders
number_of_orders = merged_df['Order ID'].nunique()
print(f"Number of Orders: {number_of_orders}")

# Calculate Average Order Value
average_order_value = total_revenue / number_of_orders
print(f"Average Order Value: ${average_order_value:,.2f}")

Total Revenue: $2,195,177.00
Total Profit: $683,960.35
Number of Orders: 1000
Average Order Value: $2,195.18


In [58]:
writer = pd.ExcelWriter('kpi_dashboard.xlsx', engine='xlsxwriter')

# Create a DataFrame for KPIs
kpi_data = {
    'KPI': ['Total Revenue', 'Total Profit', 'Number of Orders', 'Average Order Value'],
    'Value': [total_revenue, total_profit, number_of_orders, average_order_value]
}
kpi_df = pd.DataFrame(kpi_data)

# Write the KPIs DataFrame to a sheet named 'KPIs'
kpi_df.to_excel(writer, sheet_name='KPIs', index=False)

# You might want to write other dataframes (like the original sheets or the merged_df) to other sheets
# Example:
sales_df.to_excel(writer, sheet_name='Original_Sales', index=False)
merged_df.to_excel(writer, sheet_name='Merged_Data', index=False)


# Save the Excel file
writer.close()

print("\nKPIs and other data written to kpi_dashboard.xlsx")


KPIs and other data written to kpi_dashboard.xlsx


In [59]:
xls_kpi = pd.ExcelFile('kpi_dashboard.xlsx')
print("Sheets in kpi_dashboard.xlsx:")
print(xls_kpi.sheet_names)

Sheets in kpi_dashboard.xlsx:
['KPIs', 'Original_Sales', 'Merged_Data']
